# Домашнее задание 3
## Text Normalization 

deadline: 15 мая 2020, 23:30

В этом домашнем задании вы будете работать с корпусом соревнования по нормализации текстов на русском языке. 

Ссылка на соревнование:
https://www.kaggle.com/c/text-normalization-challenge-russian-language

Корпус (train-test split) доступен там же, на kaggle. Кроме того, kaggle проверяет результаты на тестовом множестве. Пример сабмита в файле: ru_sample_submission_2. 

Задача заключается в том, привести исходный текст (колонку before) в нормализованную форму (колонка after). Дополнительно известны классы токенов (колонка class), общее число классов – 15. В тестовом множестве классы токенов отсутствуют. 

Корпус состоит из предложений на русском языке и их нормализованных аналогов. Примеры продемонстрированы на kaggle.

## ПРАВИЛА
1. Домашнее задание выполняется в группе до 3-х человек.
2. Домашнее задание сдается через anytask, инвайты будут дополнительно высланы.
3. Домашнее задание оформляется в виде отчета либо в .pdf файле, либо ipython-тетрадке. 
4. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
5. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
6. Отчеты, состоящие исключительно из кода, не будут проверены и будут автоматически оценены нулевой оценкой.
7. Плагиат и любое недобросоветсное цитирование приводит к обнуление оценки. 

In [1]:
!pip install kaggle

Could not build wheels for kaggle, since package 'wheel' is not installed.
Could not build wheels for urllib3, since package 'wheel' is not installed.
Could not build wheels for six, since package 'wheel' is not installed.
Could not build wheels for certifi, since package 'wheel' is not installed.
Could not build wheels for python-dateutil, since package 'wheel' is not installed.
Could not build wheels for requests, since package 'wheel' is not installed.
Could not build wheels for tqdm, since package 'wheel' is not installed.
Could not build wheels for python-slugify, since package 'wheel' is not installed.
Could not build wheels for idna, since package 'wheel' is not installed.
Could not build wheels for chardet, since package 'wheel' is not installed.
Could not build wheels for text-unidecode, since package 'wheel' is not installed.


In [1]:
# !pip install kaggle
import os

In [2]:
os.environ['KAGGLE_USERNAME'] = 'darlitlit'
os.environ['KAGGLE_KEY'] = 'dcc30c94bda5fb17c97cc9d0f6c8e407'

In [4]:
!kaggle competitions download -c text-normalization-challenge-russian-language

text-normalization-challenge-russian-language.zip: Skipping, found more recently modified local copy (use --force to force download)


In [17]:
# !unzip text-normalization-challenge-russian-language.zip

## Часть 1. [1 балл] Эксплоративный анализ

1. Найдите примеры каждого класса и опишите, по какой логике проведена нормализация токенов разных классов. 
2. В каких случаях токены класса PLAIN подвергаются нормализации? 
3. Напишите правила для нормализации токенов класса ORDINAL. 

## Часть 2. [6 баллов]  seq2seq архитектуры
Имплементируйте несколько seq2seq архитектур. Энкодер получает на вход последовательность токенов before, декодер учится превращать их в токены after.
Энкодер и декодер работают на уровне символов, эмбеддинги символов инициализируются случайно (по аналогии с работами, в которых предложены нейросетевые модели исправления опечаток).

Эту часть задания рекомендуется выполнять с использованием allennlp (должно быть проще и удобнее).

1. [3 балла] LSTM encoder + LSTM decoder + три механизма внимания: скалярное произведение, аддитивное внимание и мультипликативное внимание (см. лекцию 6, слайд "подсчет весов attention")
2. [3 балла] Transformer

Используя автопровереку kaggle, оцените, как влияют параметры архитектуры на качество задачи.

[бонус] convolutional encoder + convolutional decoder

[бонус] pyramid LSTM (размер l+1 слоя в два раз меньше размера l, i-тый вход l+1 слоя – конкатенация выходов 2i и 2i+1)

## Часть 3. [2 балла]  Дополнительные признаки
Предложите и покажите, как можно было бы повысить качество нейросетевых моделей. Примерные варианты:
1. ансамблирование нейронных сетей
2. добавление морфологоческих признаков 
3. использование эмбеддингов слов 


## Часть 4. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Проведите анализ ошибок: когда модель ошибается? Можно ли скзаать, почему модель ошибается? Сравните результаты всех разработанных моделей. Что помогло вам в выполнении работы, чего не хватало?

# Часть 1. Эксплоративный анализ

1. Найдите примеры каждого класса и опишите, по какой логике проведена нормализация токенов разных классов. 
2. В каких случаях токены класса PLAIN подвергаются нормализации? 
3. Напишите правила для нормализации токенов класса ORDINAL. 

In [29]:
# !pip install tabulate

In [98]:
import os
import numpy as np

import pandas as pd
from tabulate import tabulate

In [99]:
train_df = pd.read_csv('ru_train.csv.zip',  dtype={'before': str, 'after': str})
train_df.before = train_df.before.astype(str)

In [100]:
train_df.head()

,sentence_id,token_id,class,before,after
0,0,0,PLAIN,По,По
1,0,1,PLAIN,состоянию,состоянию
2,0,2,PLAIN,на,на
3,0,3,DATE,1862 год,тысяча восемьсот шестьдесят второй год
4,0,4,PUNCT,.,.


In [101]:
test_df = pd.read_csv('ru_test.csv.zip',  dtype={'before': str, 'after': str})
test_df.before = test_df.before.astype(str)

In [102]:
test_df.head()

,sentence_id,token_id,before
0,0,0,Производится
1,0,1,в
2,0,2,Азии
3,0,3,","
4,0,4,Африке


In [103]:
examples = pd.read_csv('ru_sample_submission_2.csv.zip')

In [25]:
examples.head()

,id,after
0,0_0,Эта
1,0_1,книга
2,0_2,","
3,0_3,отличающаяся
4,0_4,«


## Найдите примеры каждого класса и опишите, по какой логике проведена нормализация токенов разных классов.

In [7]:
CLS = list(set(train_df['class'].values))

In [51]:
n_examples = 10
for cls in CLS:
    idx = train_df['class'] == cls
    examples = train_df[idx].sample(n_examples)
    print(tabulate(examples[['before', 'after', 'class']],
                   headers=['before', 'after', 'class'],
                   showindex=False))

before    after    class
--------  -------  -------
)         )        PUNCT
,         ,        PUNCT
)         )        PUNCT
.         .        PUNCT
)         )        PUNCT
)         )        PUNCT
«         «        PUNCT
.         .        PUNCT
,         ,        PUNCT
.         .        PUNCT
before    after    class
--------  -------  --------
-         -        VERBATIM
-         -        VERBATIM
Β         бета     VERBATIM
-         -        VERBATIM
-         -        VERBATIM
-         -        VERBATIM
-         -        VERBATIM
-         -        VERBATIM
店        店       VERBATIM
ր         ր        VERBATIM
  before  after                    class
--------  -----------------------  -------
    0185  ноль один восемь пять    DIGIT
      06  ноль шесть               DIGIT
    0941  ноль девять четыре один  DIGIT
      07  ноль семь                DIGIT
      06  ноль шесть               DIGIT
     000  ноль ноль ноль           DIGIT
    0341  ноль три четыре один     DI

before            after                                                                                                        class
----------------  -----------------------------------------------------------------------------------------------------------  ----------
vishime.ru        в_trans и_trans ш_trans и_trans м_trans точка р_trans у_trans                                                ELECTRONIC
tolweb.org        т_trans о_trans л_trans у_trans э_trans б_trans точка о_trans р_trans г_trans                                ELECTRONIC
Shakira.com       ш_trans е_trans й_trans к_trans и_trans р_trans а_trans точка к_trans о_trans м_trans                        ELECTRONIC
OM.net            о_trans м_trans точка н_trans е_trans т_trans                                                                ELECTRONIC
Sportal.bg        с_trans п_trans о_trans р_trans т_trans а_trans л_trans точка b g                                            ELECTRONIC
www.enric.es      w w w точка э_trans н

'PUNCT' -- знаки пунктуация, никак не меняются

'VERBATIM' -- как вслух произносится символ ("в квадрате", "нижнее подчеркивание", буквы греческого алфавита)

'DIGIT' -- числа

'MEASURE' -- как вслух произносятся сокращения для единиц измерения

'TELEPHONE' -- телефон, вместо - ставится sil

'LETTERS' -- пробелом разделяются аббревиатуры

'PLAIN' -- слова, написанные на русском, не меняются, иначе -- транскрибируются

'DECIMAL' -- числа, дробные и с сокращениями (тыс, млн)

'ORDINAL' -- порядковые числительные

'FRACTION' -- дроби

'ELECTRONIC' -- как произносятся ссылки (транскрипция)

'DATE' -- даты

'MONEY' -- числа и валютные названия (транскрипцией, сша -- сэ ш а)

'TIME' -- время

'CARDINAL' -- числа в именительном падеже

## В каких случаях токены класса PLAIN подвергаются нормализации?

In [52]:
plain = train_df[train_df['class'] == 'PLAIN']
plain[plain.before != plain.after].head()

,sentence_id,token_id,class,before,after
43,3,7,PLAIN,Tiberius,т_trans и_trans б_trans е_trans р_trans и_tran...
44,3,8,PLAIN,Julius,д_trans ж_trans у_trans л_trans и_trans у_tran...
45,3,9,PLAIN,Pollienus,п_trans о_trans л_trans л_trans и_trans е_tran...
46,3,10,PLAIN,Auspex,о_trans с_trans п_trans е_trans к_trans с_trans
153,10,4,PLAIN,Half,х_trans а_trans л_trans ф_trans


'PLAIN' нормализуются (транслитерируются), если написаны нерусскими буквами

## Напишите правила для нормализации токенов класса ORDINAL

In [64]:
ordinal = train_df[train_df['class'] == 'ORDINAL']
ordinal.head()

,sentence_id,token_id,class,before,after
53,3,17,ORDINAL,III,третьего
163,11,1,ORDINAL,1895,тысяча восемьсот девяносто пятом
164,11,2,ORDINAL,—1896,тысяча восемьсот девяносто шестом
485,35,5,ORDINAL,1796,тысяча семьсот девяносто шестого
487,35,7,ORDINAL,1802,тысяча восемьсот второй


In [65]:
with_ordinals = list(set(ordinal['sentence_id'].values))
with_ordinals = train_df[train_df['sentence_id'].isin(with_ordinals)]

In [74]:
before_df = with_ordinals.groupby(['sentence_id'])['before'].apply(list)
after_df = with_ordinals.groupby(['sentence_id'])['after'].apply(list)

In [76]:
before_df = pd.DataFrame({'text': before_df}).reset_index()
before_df = before_df.text.apply(lambda x: ' '.join(x))

In [84]:
after_df = pd.DataFrame({'text': after_df}).reset_index()
after_df = after_df.text.apply(lambda x: ' '.join(x))

In [87]:
before_df

0        Тиберий Юлий Поллиен Ауспекс ( лат . Tiberius ...
1        в 1895 —1896 годах служил на Черноморском флот...
2        После Павловских реформ , с 1796 по 1802 год ,...
3        Некоторые источники указывают на период 1109 —...
4        Джон Стюарт , 1-й маркиз Бьют , стал носить эт...
                               ...                        
33236    Сформирована 21 апреля 1944 года путем преобра...
33237    Соответственно отодвигался и ввод в прорыв 6-г...
33238    В первом тайме на 7-й минуте Зинедин Зидан реа...
33239    Packard Station Sedan — это легковой автомобил...
33240    В отличие от зенитных артиллерийских полков в ...
Name: text, Length: 33241, dtype: object

## Часть 2. [6 баллов]  seq2seq архитектуры
Имплементируйте несколько seq2seq архитектур. Энкодер получает на вход последовательность токенов before, декодер учится превращать их в токены after.
Энкодер и декодер работают на уровне символов, эмбеддинги символов инициализируются случайно (по аналогии с работами, в которых предложены нейросетевые модели исправления опечаток).

Эту часть задания рекомендуется выполнять с использованием allennlp (должно быть проще и удобнее).

1. [3 балла] LSTM encoder + LSTM decoder + три механизма внимания: скалярное произведение, аддитивное внимание и мультипликативное внимание (см. лекцию 6, слайд "подсчет весов attention")
2. [3 балла] Transformer

Используя автопровереку kaggle, оцените, как влияют параметры архитектуры на качество задачи.

[бонус] convolutional encoder + convolutional decoder

[бонус] pyramid LSTM (размер l+1 слоя в два раз меньше размера l, i-тый вход l+1 слоя – конкатенация выходов 2i и 2i+1)

### LSTM encoder + LSTM decoder + три механизма внимания: скалярное произведение, аддитивное внимание и мультипликативное внимание (см. лекцию 6, слайд "подсчет весов attention")

Скалярное произведение -- allennlp.modules.attention.dot_product_attention.DotProductAttention


Аддитивное внимание -- allennlp.modules.attention.additive_attention.AdditiveAttention


Мультипликативное внимание -- allennlp.modules.attention.bilinear_attention.BilinearAttention

In [10]:
# !pip install allennlp

In [1]:
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.predictors import SimpleSeq2SeqPredictor
from allennlp.modules.attention import LinearAttention
from allennlp.nn.activations import Activation
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.token_indexers import TokenCharactersIndexer
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data import Instance
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.modules.attention import LinearAttention, BilinearAttention, DotProductAttention, AdditiveAttention
from allennlp.modules.seq2seq_encoders import StackedSelfAttentionEncoder
from allennlp.data.tokenizers.word_tokenizer import WordTokenizer
from allennlp.data.tokenizers.token import Token
from allennlp.data.tokenizers.word_splitter import WordSplitter
from allennlp.data.tokenizers.tokenizer import Tokenizer

import torch

import torch.optim as optim

from typing import List

from overrides import overrides

from tqdm.notebook import tqdm_notebook as tqdm

from sklearn.model_selection import train_test_split

import pickle

import itertools

In [2]:
CUDA_DEVICE = torch.cuda.current_device()

In [3]:
assert torch.cuda.device_count() > 0

In [30]:
# CUDA_DEVICE = torch.device("cuda")

In [23]:
# sentence_df = train_df.groupby(['sentence_id'])['after'].apply(list)

In [24]:
# sentence_df.head()
# 

In [7]:
# sentence_df = pd.DataFrame({'text': sentence_df}).reset_index()

In [8]:
# sentence_df.text = sentence_df.text.apply(lambda x: ' '.join(map(str, x)))

In [9]:
# sentence_df.to_csv('after_train.csv')

In [10]:
len(sentence_df)

NameError: name 'sentence_df' is not defined

In [33]:
def group_by_sentence(df: pd.DataFrame):
#     last_sentence_id = None
#     new_data = []
#     before = []
#     after = []
#     for i, row in tqdm(df.iterrows(),total=len(df)):
#         if last_sentence_id and row.sentence_id != last_sentence_id:
#             new_data.append([last_sentence_id, ' '.join(before), ' '.join(after)])
#         before.append(row.before)
#         after.append(row.after)
#         last_sentence_id = row.sentence_id
    
#     if last_sentence_id and row.sentence_id != last_sentence_id:
#         new_data.append([last_sentence_id, ' '.join(before), ' '.join(after)])
    
    new_df = pd.DataFrame(columns=['before', 'after'])
    groupped = df.groupby(['sentence_id'])
    new_df['after'] = groupped['after'].apply(lambda x: ' '.join(x))
    new_df['before'] = groupped['before'].apply(lambda x: ' '.join(x))

    return new_df

In [16]:
group_by_sentence(train_df.head(1000))

,before,after
sentence_id,,
0,По состоянию на 1862 год .,По состоянию на тысяча восемьсот шестьдесят вт...
1,Оснащались латными рукавицами и сабатонами с н...,Оснащались латными рукавицами и сабатонами с н...
2,"В конце 1811 года , вследствие конфликта с про...","В конце тысяча восемьсот одиннадцатого года , ..."
3,Тиберий Юлий Поллиен Ауспекс ( лат . Tiberius ...,Тиберий Юлий Поллиен Ауспекс ( лат . т_trans и...
4,"Севернее Дудинки и северо - восточнее Белочи ,...","Севернее Дудинки и северо - восточнее Белочи ,..."
...,...,...
67,"Джон Стюарт , 1-й маркиз Бьют , стал носить эт...","Джон Стюарт , первый маркиз Бьют , стал носить..."
68,На Плоском Мире восьмисторонняя призма способн...,На Плоском Мире восьмисторонняя призма способн...
69,"В 2011 вышел четвертый альбом , Je suis , в 20...",В двух тысячах одиннадцати вышел четвертый аль...


In [17]:
train_df_groupped = group_by_sentence(train_df)

In [19]:
with open('train_df_groupped.pickle', 'wb') as f:
    pickle.dump(train_df_groupped, f)

In [29]:
with open('train_df_groupped.pickle', 'rb') as f:
    train_df_groupped = pickle.load(f)

In [12]:
len(train_df_groupped[train_df_groupped['before'] != train_df_groupped['after']]), len(train_df_groupped)

(438263, 761436)

In [13]:
train_cut = train_df_groupped[train_df_groupped['before'] != train_df_groupped['after']]
train_cut = train_cut[train_cut['before'].apply(len) < 500]
train_cut = train_cut[train_cut['after'].apply(len) < 500]

In [14]:
len(train_cut)

427943

In [15]:
_X_train, _X_test = train_test_split(train_cut, test_size=0.1, random_state=42)


In [16]:
_X_pretrain = _X_train.copy()
_X_pretrain.after = _X_pretrain.before

In [18]:
sorted(train_cut.before.apply(len))[-10:]

[336, 346, 354, 359, 382, 397, 404, 412, 438, 473]

In [113]:
_X_pretrain.to_csv('X_pretrain_allennlp.tsv', sep='\t', header=False, columns=['before', 'after'], index=False)
_X_train.to_csv('X_train_allennlp.tsv', sep='\t', header=False, columns=['before', 'after'], index=False)
_X_test.to_csv('X_test_allennlp.tsv', sep='\t', header=False, columns=['before', 'after'], index=False)

In [4]:
tokenizer = CharacterTokenizer(byte_encoding=None, lowercase_characters=True)
source_token_indexer =  {
    'tokens': SingleIdTokenIndexer(namespace='tokens')
}
target_token_indexer =  {
    'tokens': SingleIdTokenIndexer(namespace='target_tokens')
}
reader = Seq2SeqDatasetReader(
        source_tokenizer=CharacterTokenizer(end_tokens=['<eos>']),
        target_tokenizer=CharacterTokenizer(start_tokens=['<sos>']),
        source_token_indexers=source_token_indexer,
        target_token_indexers=target_token_indexer,
#         lazy=True
)
X_train = reader.read('X_train_allennlp.tsv')
X_pretrain = reader.read('X_pretrain_allennlp.tsv')
X_test = reader.read('X_test_allennlp.tsv')
# X_train = reader.read('X_train_allennlp_10000.tsv')
# X_pretrain = reader.read('X_pretrain_allennlp_10000.tsv')
# X_test = reader.read('X_test_allennlp_10000.tsv')

385148it [02:13, 2892.18it/s]
385148it [01:50, 3482.74it/s]
42795it [00:07, 5431.98it/s]


In [5]:
print(X_pretrain[2])

Instance with fields:
 	 source_tokens: TextField of length 85 with text: 
 		[@start@, Р, а, б, о, т, а, м, и,  , р, у, к, о, в, о, д, и, л, и,  , А, .,  , И, .,  , С, о, к, о,
		л, о, в,  , и,  , е, г, о,  , з, а, м, е, с, т, и, т, е, л, и,  , Г, .,  , А, .,  , Т, ю, л, и, н,
		, и,  , Ю, .,  , А, .,  , М, о, з, ж, о, р, и, н,  , ., <eos>, @end@]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 target_tokens: TextField of length 85 with text: 
 		[@start@, <sos>, Р, а, б, о, т, а, м, и,  , р, у, к, о, в, о, д, и, л, и,  , А, .,  , И, .,  , С, о,
		к, о, л, о, в,  , и,  , е, г, о,  , з, а, м, е, с, т, и, т, е, л, и,  , Г, .,  , А, .,  , Т, ю, л,
		и, н,  , и,  , Ю, .,  , А, .,  , М, о, з, ж, о, р, и, н,  , ., @end@]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 



In [106]:
print(X_test[2])

Instance with fields:
 	 source_tokens: TextField of length 96 with text: 
 		[@start@, «,  , К, р, а, с, н, о,  , -,  , ч, е, р, н, ы, е,  , »,  , п, р, о, п, у, с, т, и, л, и,
		, 1, 2,  , г, о, л, о, в,  , ,,  , 4,  , и, з,  , н, и, х,  , б, ы, л, и,  , п, р, о, п, у, щ, е, н,
		ы,  , в,  , о, т, с, у, т, с, т, в, и, и,  , Д, ж, о, н, а,  , н, а,  , п, о, л, е,  , ., @end@]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 target_tokens: TextField of length 109 with text: 
 		[@start@, «,  , К, р, а, с, н, о,  , -,  , ч, е, р, н, ы, е,  , »,  , п, р, о, п, у, с, т, и, л, и,
		, д, в, е, н, а, д, ц, а, т, ь,  , г, о, л, о, в,  , ,,  , ч, е, т, ы, р, е,  , и, з,  , н, и, х,  ,
		б, ы, л, и,  , п, р, о, п, у, щ, е, н, ы,  , в,  , о, т, с, у, т, с, т, в, и, и,  , Д, ж, о, н, а,
		, н, а,  , п, о, л, е,  , ., @end@]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 



In [5]:
vocab = Vocabulary.from_instances(X_train + X_test,
                                  min_count={'source_tokens': 10,
                                             'target_tokens': 10})

100%|██████████| 20000/20000 [00:02<00:00, 8910.14it/s]


In [24]:
vocab

Vocabulary with namespaces:  target_tokens, Size: 391 || tokens, Size: 2180 || Non Padded Namespaces: {'*labels', '*tags'}

In [6]:
VOCAB_DIR = "vocabulary"

In [8]:

vocab.save_to_files(VOCAB_DIR)

In [7]:
vocab = Vocabulary.from_files(VOCAB_DIR)

In [66]:
vocab.get_vocab_size('tokens')

37964

In [5]:
CUDA_DEVICE = 0
CUDA_DEVICES = list(range(8))
EN_EMBEDDING_DIM = 256
HIDDEN_DIM = 256
max_decoding_steps = 10
ZH_EMBEDDING_DIM = 256

In [30]:


try:
    del model
except:
    pass

try:
    del trainer
except:
    pass
try:
    del pretrainer
except:
    pass
    

torch.cuda.empty_cache()

In [8]:
def get_model(*, vocab):
    en_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                             embedding_dim=EN_EMBEDDING_DIM)
    # encoder = PytorchSeq2SeqWrapper(
    #     torch.nn.LSTM(EN_EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
    encoder = StackedSelfAttentionEncoder(input_dim=EN_EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, projection_dim=128, feedforward_hidden_dim=128, num_layers=1, num_attention_heads=8)

    source_embedder = BasicTextFieldEmbedder({"tokens": en_embedding})

    # attention = LinearAttention(HIDDEN_DIM, HIDDEN_DIM, activation=Activation.by_name('tanh')())
    # attention = BilinearAttention(HIDDEN_DIM, HIDDEN_DIM)
    attention = DotProductAttention()

    max_decoding_steps = 400
    model = SimpleSeq2Seq(vocab, source_embedder, encoder, max_decoding_steps,
                          target_embedding_dim=ZH_EMBEDDING_DIM,
                          target_namespace='target_tokens',
                          attention=attention,
                          beam_size=8,
                          use_bleu=True)
    model.cuda(CUDA_DEVICE)
    
    return model


def get_trainer(*, model, X_train, X_test):
   
    optimizer = optim.Adam(model.parameters())
    iterator = BucketIterator(batch_size=1024, sorting_keys=[("source_tokens", "num_tokens")])

    iterator.index_with(vocab)

    trainer = Trainer(model=model,
                      optimizer=optimizer,
                      iterator=iterator,
                      train_dataset=X_train,
                      validation_dataset=X_test,
                      num_epochs=1,
                      cuda_device=CUDA_DEVICES)

    return trainer

def run_trainer(*, model,  trainer, epochs):
    for i in range(epochs):
        print('Epoch: {}'.format(i))
        trainer.train()

        predictor = SimpleSeq2SeqPredictor(model, reader)

        for instance in itertools.islice(X_test, 10):
            print('SOURCE:', ''.join(map(str, instance.fields['source_tokens'].tokens)))
            print('GOLD:', ''.join(map(str, instance.fields['target_tokens'].tokens)))
            print('PRED:', ''.join(map(str, predictor.predict_instance(instance)['predicted_tokens'])))
        print()
    return trainer

In [9]:
model = get_model(vocab=vocab)

In [10]:
pretrainer = get_trainer(model=model, X_train=X_pretrain, X_test=X_test[:100])
run_trainer(model=model, trainer=pretrainer, epochs=200)

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


Epoch: 0


loss: 3.9665 ||: 100%|██████████| 48/48 [03:25<00:00,  4.29s/it]  
BLEU: 0.0000, loss: 3.7895 ||: 100%|██████████| 1/1 [00:05<00:00,  5.16s/it]


SOURCE: @start@Проходило 25 октября 2009 года на « Сивик - арене » в Питтсбурге , Пенсильвания .<eos>@end@
GOLD: @start@<sos>Проходило двадцать пятого октября две тысячи девятого года на « Сивик - арене » в Питтсбурге , Пенсильвания .@end@
PRED: <sos>В                                                                                                                                                                                                                                                                                                                                                                                                              
SOURCE: @start@8 в . ( от 9 — 8 вв . ) — Возникновение греческого алфавита .<eos>@end@
GOLD: @start@<sos>восемь в . ( от девять до восемь в в . ) — Возникновение греческого алфавита .@end@
PRED: <sos>В                                                                                                                                                    

loss: 3.3434 ||: 100%|██████████| 48/48 [01:42<00:00,  2.14s/it]
BLEU: 0.0000, loss: 3.2058 ||: 100%|██████████| 1/1 [00:05<00:00,  5.39s/it]


...

SOURCE: @start@Проходило 25 октября 2009 года на « Сивик - арене » в Питтсбурге , Пенсильвания .<eos>@end@
GOLD: @start@<sos>Проходило двадцать пятого октября две тысячи девятого года на « Сивик - арене » в Питтсбурге , Пенсильвания .@end@
PRED: <sos>Проходило 25 октября 2011 ) .
SOURCE: @start@8 в . ( от 9 — 8 вв . ) — Возникновение греческого алфавита .<eos>@end@
GOLD: @start@<sos>восемь в . ( от девять до восемь в в . ) — Возникновение греческого алфавита .@end@
PRED: <sos>8 в . ( от 9 — 1998 . Архивирование протительского поставляет в составляет .
SOURCE: @start@« Красно - черные » пропустили 12 голов , 4 из них были пропущены в отсутствии Джона на поле .<eos>@end@
GOLD: @start@<sos>« Красно - черные » пропустили двенадцать голов , четыре из них были пропущены в отсутствии Джона на поле .@end@
PRED: <sos>« Красно - человерского из первоисточника 2013 . Архивировано из первоисточника 2011 года .
SOURCE: @start@Проверено 27 мая 2012 . Архивировано из первоисточника 23 сентября 2012 .

loss: 1.6230 ||: 100%|██████████| 48/48 [01:40<00:00,  2.09s/it]
BLEU: 0.1220, loss: 3.1180 ||: 100%|██████████| 1/1 [00:05<00:00,  5.15s/it]


SOURCE: @start@Проходило 25 октября 2009 года на « Сивик - арене » в Питтсбурге , Пенсильвания .<eos>@end@
GOLD: @start@<sos>Проходило двадцать пятого октября две тысячи девятого года на « Сивик - арене » в Питтсбурге , Пенсильвания .@end@
PRED: <sos>Проходило 25 октября 2012 ) .
SOURCE: @start@8 в . ( от 9 — 8 вв . ) — Возникновение греческого алфавита .<eos>@end@
GOLD: @start@<sos>восемь в . ( от девять до восемь в в . ) — Возникновение греческого алфавита .@end@
PRED: <sos>8 в . ( от 9 — 1978 . Архивирование предительственный компании .
SOURCE: @start@« Красно - черные » пропустили 12 голов , 4 из них были пропущены в отсутствии Джона на поле .<eos>@end@
GOLD: @start@<sos>« Красно - черные » пропустили двенадцать голов , четыре из них были пропущены в отсутствии Джона на поле .@end@
PRED: <sos>« Красно - черника в составляется в составляется в составляется в составляется в составляется в составляется в составляется .
SOURCE: @start@Проверено 27 мая 2012 . Архивировано из первоисточн

loss: 1.6020 ||: 100%|██████████| 48/48 [01:40<00:00,  2.09s/it]
BLEU: 0.1369, loss: 3.1232 ||: 100%|██████████| 1/1 [00:05<00:00,  5.46s/it]


SOURCE: @start@Проходило 25 октября 2009 года на « Сивик - арене » в Питтсбурге , Пенсильвания .<eos>@end@
GOLD: @start@<sos>Проходило двадцать пятого октября две тысячи девятого года на « Сивик - арене » в Питтсбурге , Пенсильвания .@end@
PRED: <sos>Проходило 25 октября 2012 ) .
SOURCE: @start@8 в . ( от 9 — 8 вв . ) — Возникновение греческого алфавита .<eos>@end@
GOLD: @start@<sos>восемь в . ( от девять до восемь в в . ) — Возникновение греческого алфавита .@end@
PRED: <sos>8 в . ( от 19 — 1978 . Архивирование призверской компании в 1945 года .
SOURCE: @start@« Красно - черные » пропустили 12 голов , 4 из них были пропущены в отсутствии Джона на поле .<eos>@end@
GOLD: @start@<sos>« Красно - черные » пропустили двенадцать голов , четыре из них были пропущены в отсутствии Джона на поле .@end@
PRED: <sos>« Красно - черный компания , в составляет в составляется в составляет в составляется в составляется в составляется в составляется в составляется .
SOURCE: @start@Проверено 27 мая 2012 .

In [11]:
trainer = get_trainer(model=model, X_train=X_train, X_test=X_test[:100])
run_trainer(model=model, trainer=trainer, epochs=200)

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


Epoch: 0


loss: 1.7768 ||: 100%|██████████| 48/48 [05:24<00:00,  6.76s/it]  
BLEU: 0.1393, loss: 1.4236 ||: 100%|██████████| 1/1 [00:05<00:00,  5.23s/it]


SOURCE: @start@Проходило 25 октября 2009 года на « Сивик - арене » в Питтсбурге , Пенсильвания .<eos>@end@
GOLD: @start@<sos>Проходило двадцать пятого октября две тысячи девятого года на « Сивик - арене » в Питтсбурге , Пенсильвания .@end@
PRED: <sos>Проходило года .
SOURCE: @start@8 в . ( от 9 — 8 вв . ) — Возникновение греческого алфавита .<eos>@end@
GOLD: @start@<sos>восемь в . ( от девять до восемь в в . ) — Возникновение греческого алфавита .@end@
PRED: <sos>дев . ) .
SOURCE: @start@« Красно - черные » пропустили 12 голов , 4 из них были пропущены в отсутствии Джона на поле .<eos>@end@
GOLD: @start@<sos>« Красно - черные » пропустили двенадцать голов , четыре из них были пропущены в отсутствии Джона на поле .@end@
PRED: <sos>« Красного года .
SOURCE: @start@Проверено 27 мая 2012 . Архивировано из первоисточника 23 сентября 2012 . Кодваль ( фр . ) .<eos>@end@
GOLD: @start@<sos>Проверено двадцать седьмого мая две тысячи двенадцатого года . Архивировано из первоисточника двадцать тре

loss: 1.3453 ||: 100%|██████████| 48/48 [04:03<00:00,  5.08s/it]
BLEU: 0.1005, loss: 1.2894 ||: 100%|██████████| 1/1 [00:05<00:00,  5.54s/it]


...

SOURCE: @start@Проходило 25 октября 2009 года на « Сивик - арене » в Питтсбурге , Пенсильвания .<eos>@end@
GOLD: @start@<sos>Проходило двадцать пятого октября две тысячи девятого года на « Сивик - арене » в Питтсбурге , Пенсильвания .@end@
PRED: <sos>Проходило двадцать седьмого января две тысячи тринадцатого года . « Советственность на процентов , поставляется в тысяча девятьсот шестьдесят седьмого года , поставляется в тысяча девятьсот пятьдесят седьмом году .
SOURCE: @start@8 в . ( от 9 — 8 вв . ) — Возникновение греческого алфавита .<eos>@end@
GOLD: @start@<sos>восемь в . ( от девять до восемь в в . ) — Возникновение греческого алфавита .@end@
PRED: <sos>восемнадцатое апреля тысяча восемьсот восемьдесят втором году вв . ) — тысяча восемьсот восемьдесят второго года .
SOURCE: @start@« Красно - черные » пропустили 12 голов , 4 из них были пропущены в отсутствии Джона на поле .<eos>@end@
GOLD: @start@<sos>« Красно - черные » пропустили двенадцать голов , четыре из них были пропущены в 

loss: 0.8756 ||: 100%|██████████| 48/48 [04:04<00:00,  5.08s/it]
BLEU: 0.3695, loss: 0.8279 ||: 100%|██████████| 1/1 [00:07<00:00,  7.75s/it]


SOURCE: @start@Проходило 25 октября 2009 года на « Сивик - арене » в Питтсбурге , Пенсильвания .<eos>@end@
GOLD: @start@<sos>Проходило двадцать пятого октября две тысячи девятого года на « Сивик - арене » в Питтсбурге , Пенсильвания .@end@
PRED: <sos>Проходило двадцать седьмого ноября две тысячи тринадцатого года . « составительность населенного составителенного составительного составительного составительности .
SOURCE: @start@8 в . ( от 9 — 8 вв . ) — Возникновение греческого алфавита .<eos>@end@
GOLD: @start@<sos>восемь в . ( от девять до восемь в в . ) — Возникновение греческого алфавита .@end@
PRED: <sos>восемнадцатое апреля тысяча девятьсот восемьдесят восьмого года , тысяча восемьсот восемьдесят две тысячи восемьсот восемьдесят восемьсот восемьдесят восемь . — две тысячи восемьсот восемьдесят девятого года .
SOURCE: @start@« Красно - черные » пропустили 12 голов , 4 из них были пропущены в отсутствии Джона на поле .<eos>@end@
GOLD: @start@<sos>« Красно - черные » пропустили двена

loss: 1.2722 ||: 100%|██████████| 48/48 [04:04<00:00,  5.09s/it]
BLEU: 0.2457, loss: 1.0690 ||: 100%|██████████| 1/1 [00:05<00:00,  5.64s/it]


SOURCE: @start@Проходило 25 октября 2009 года на « Сивик - арене » в Питтсбурге , Пенсильвания .<eos>@end@
GOLD: @start@<sos>Проходило двадцать пятого октября две тысячи девятого года на « Сивик - арене » в Питтсбурге , Пенсильвания .@end@
PRED: <sos>Проходило четырнадцать восемьдесят одиннадцать восемьдесят одиннадцать восемь sil две тысячи девятьсот девяносто восемьдесят восемьдесят одиннадцать семьдесят двадцать четыре — тысяча девятьсот двадцать четыре — тысяча девятьсот двадцать четыре — тысяча девятьсот двадцать четыре — тысяча девятьсот двадцать четыре — тысяча девятьсот двадцать пятого года .
SOURCE: @start@8 в . ( от 9 — 8 вв . ) — Возникновение греческого алфавита .<eos>@end@
GOLD: @start@<sos>восемь в . ( от девять до восемь в в . ) — Возникновение греческого алфавита .@end@
PRED: <sos>в тысяча девятьсот девяносто восемьдесят двадцать четвертого восемьдесят одиннадцать четыре — тысяча девятьсот двадцать четыре — тысяча девятьсот двадцать четыре — тысяча девятьсот девяносто в

In [12]:
# Here's how to save the model.
with open("./models/simple_sentence_lstm_200epochs/simple_sentence_lstm_200epochs.th", 'wb') as f:
    torch.save(model.state_dict(), f)

In [4]:
@WordSplitter.register('character_with_trans')
class CharacterWithTransWordSplitter(WordSplitter):
    """
    A ``WordSplitter`` that convert string to characters except of a_trans b_trans and so on cases
    """
    def __init__(
        self,
        use_trans_tokens=True, 
        lowercase_characters: bool = False
    ) -> None:
        self._use_trans_tokens = use_trans_tokens
        self._lowercase_characters = lowercase_characters
    
    @overrides
    def batch_split_words(self, sentences: List[str]) -> List[List[Token]]:
        return [self.split_words(setence)
                for sentence in sentences]
    
    def _sanitize(self, sentence: List[str]) -> List[Token]:
        return list(map(Token, sentence))

    @overrides
    def split_words(self, sentence: str) -> List[Token]:
        # This works because our Token class matches spacy's.
        if self._lowercase_characters:
            sentence = sentence.lower()
        words = sentence.split(' ')
        out = []
        for word in words:
            if self._use_trans_tokens and word.endswith('trans'):
                out.append(word)
            else:
                out += list(word)
            out.append(' ')
        out.pop()
        
        return self._sanitize(out)

In [16]:
source_tokenizer = WordTokenizer(
    word_splitter=CharacterWithTransWordSplitter(lowercase_characters=False), 
    end_tokens=['<eos>'],
)
target_tokenizer = WordTokenizer(
    word_splitter=CharacterWithTransWordSplitter(lowercase_characters=False),
    start_tokens=['<sos>'],
)
source_token_indexer =  {
    'tokens': SingleIdTokenIndexer(namespace='tokens')
}
target_token_indexer =  {
    'tokens': SingleIdTokenIndexer(namespace='target_tokens')
}
reader = Seq2SeqDatasetReader(
    source_tokenizer=source_tokenizer,
    target_tokenizer=target_tokenizer,
#         source_tokenizer=CharacterTokenizer(),
#         target_tokenizer=CharacterTokenizer(start_tokens=['<sos>']),
        source_token_indexers=source_token_indexer,
        target_token_indexers=target_token_indexer,
#         lazy=True
)
# X_train = reader.read('X_train_allennlp.tsv')
# X_pretrain = reader.read('X_pretrain_allennlp.tsv')
# X_test = reader.read('X_test_allennlp.tsv')
X_train = reader.read('X_train_allennlp_10000.tsv')
X_pretrain = reader.read('X_pretrain_allennlp_10000.tsv')
X_test = reader.read('X_test_allennlp_10000.tsv')

10000it [00:02, 4537.78it/s]
10000it [00:01, 5339.54it/s]
10000it [00:02, 4503.64it/s]


In [17]:
print(''.join(map(str, X_train[0]['source_tokens'])))
print(''.join(map(str, X_train[0]['target_tokens'])))

@start@Получил статус села ( Pueblo ) 19 августа 1926 года .<eos>@end@
@start@<sos>Получил статус села ( п_trans у_trans э_trans б_trans л_trans о_trans ) девятнадцатого августа тысяча девятьсот двадцать шестого года .@end@


Напишем процессинг предложений

Процессинг устроен следующим образом -- берем участки, где токены before не равны токенам в after
от таких участков берем **left_window_size** токенов слева и **right_windiw_size** токенов справа

Причем если есть пересекающиеся такие кусочки, их склеиваем.

Опционально (**_add_unchanged**) все пропущенные токены тоже оставляем, но добавляем пометку -- $different == False$

В итоге предложения разбиваются на непересекающиеся куски (окна), где
- **different == True** можно подавать на вход Trainer.
- **different = False** в предикте используем как неизмененную часть


In [18]:
import re

In [21]:
class WindowMaker:
    need_transform_regex = re.compile('[A-Za-z0-9]')
    
    def __init__(
        self,
        add_unchanged=False,
        left_window_size=2, 
        right_window_size=2,
    ):
        self._add_unchanged = add_unchanged
        self._left_window_size = left_window_size
        self._right_window_size = right_window_size
    
    def __call__(
        self, 
        df: pd.DataFrame,
        mode='train',
    ) -> pd.DataFrame:
        groupped = df.groupby(['sentence_id'])
        return groupped.apply(lambda x: self._process_sentence(x, mode))
    
    def _need_transform(self, a, b, mode):
        if mode == 'train':
            return a != b
        upcase_cnt = sum(1 for c in a if c.isupper())
        if upcase_cnt > 2:
            return True
        if WindowMaker.need_transform_regex.search(a):
            return True
        return False
    
    def _process_sentence(self, df: pd.DataFrame, mode) -> pd.DataFrame:
        new_before = []
        new_after = []

        before = df.before.to_list()
        if mode == 'test':
            after = before
        else:
            after = df.after.to_list()
        assert len(before) == len(after)
        i = 0
        different = []
        left_window_idx = 0
        right_window_idx = 0
        while i < len(before):
            start_window = i
            while i < len(before) and not self._need_transform(before[i], after[i], mode):
                i += 1
                
            left_window_idx = max(0, i - self._left_window_size)
            any_dirretent = False
            while i < len(before) and self._need_transform(before[i], after[i], mode):
                any_dirretent = True
                i += 1
                
            right_window_idx = min(len(before), i + self._right_window_size)
            
            
            if not any_dirretent:
                left_window_idx = right_window_idx = i
            
            i = right_window_idx
            
            if self._add_unchanged and start_window < left_window_idx:
                new_before.append(' '.join(before[start_window:left_window_idx]))
                new_after.append(' '.join(after[start_window:left_window_idx]))
                different.append(False)
                
            if any_dirretent:
                new_before.append(' '.join(before[left_window_idx:right_window_idx]))
                new_after.append(' '.join(after[left_window_idx:right_window_idx]))
                different.append(True)
        if self._add_unchanged:
            return pd.DataFrame({
                'before': new_before, 
                'after': new_after,
                'different': different,
            })
        return pd.DataFrame({
            'before': new_before, 
            'after': new_after,
        })
            

In [31]:
WindowMaker(add_unchanged=False)(train_df.head(100))

before  \
sentence_id                                                 
0           0                     состоянию на 1862 год .   
2           0              В конце 1811 года , вследствие   
3           0  лат . Tiberius Julius Pollienus Auspex ) —   
            1                   деятель начала III века .   
6           0    Проверено 12 февраля 2013 . Архивировано   
            1      из первоисточника 15 февраля 2013 . TV   

                                                           after  
sentence_id                                                       
0           0  состоянию на тысяча восемьсот шестьдесят второ...  
2           0  В конце тысяча восемьсот одиннадцатого года , ...  
3           0  лат . т_trans и_trans б_trans е_trans р_trans ...  
            1                     деятель начала третьего века .  
6           0  Проверено двенадцатого февраля две тысячи трин...  
            1  из первоисточника пятнадцатого февраля две тыс...

Проверим, что никакие кусочки не были потеряны:

In [35]:
def test():
    # группируем исходные данные
    df1 = group_by_sentence(train_df.head(100))
    # группироуем окошки
    df2 = group_by_sentence(WindowMaker(add_unchanged=True)(train_df.head(100)))
    assert(len(df1.before) == len(df2.before))
    assert(len(df1.after) == len(df2.after))
    assert df1[df1.before != df2.before].empty
    assert df1[df1.after != df2.after].empty
    

test()

На всякий случай смотрим глазами:

In [38]:
group_by_sentence(train_df.head(1000))

,before,after
sentence_id,,
0,По состоянию на 1862 год .,По состоянию на тысяча восемьсот шестьдесят вт...
1,Оснащались латными рукавицами и сабатонами с н...,Оснащались латными рукавицами и сабатонами с н...
2,"В конце 1811 года , вследствие конфликта с про...","В конце тысяча восемьсот одиннадцатого года , ..."
3,Тиберий Юлий Поллиен Ауспекс ( лат . Tiberius ...,Тиберий Юлий Поллиен Ауспекс ( лат . т_trans и...
4,"Севернее Дудинки и северо - восточнее Белочи ,...","Севернее Дудинки и северо - восточнее Белочи ,..."
...,...,...
67,"Джон Стюарт , 1-й маркиз Бьют , стал носить эт...","Джон Стюарт , первый маркиз Бьют , стал носить..."
68,На Плоском Мире восьмисторонняя призма способн...,На Плоском Мире восьмисторонняя призма способн...
69,"В 2011 вышел четвертый альбом , Je suis , в 20...",В двух тысячах одиннадцати вышел четвертый аль...


In [39]:
group_by_sentence(WindowMaker(add_unchanged=True)(train_df.head(1000)))

,before,after
sentence_id,,
0,По состоянию на 1862 год .,По состоянию на тысяча восемьсот шестьдесят вт...
1,Оснащались латными рукавицами и сабатонами с н...,Оснащались латными рукавицами и сабатонами с н...
2,"В конце 1811 года , вследствие конфликта с про...","В конце тысяча восемьсот одиннадцатого года , ..."
3,Тиберий Юлий Поллиен Ауспекс ( лат . Tiberius ...,Тиберий Юлий Поллиен Ауспекс ( лат . т_trans и...
4,"Севернее Дудинки и северо - восточнее Белочи ,...","Севернее Дудинки и северо - восточнее Белочи ,..."
...,...,...
67,"Джон Стюарт , 1-й маркиз Бьют , стал носить эт...","Джон Стюарт , первый маркиз Бьют , стал носить..."
68,На Плоском Мире восьмисторонняя призма способн...,На Плоском Мире восьмисторонняя призма способн...
69,"В 2011 вышел четвертый альбом , Je suis , в , ...",В двух тысячах одиннадцати вышел четвертый аль...


Проверяем режим для тестовой выборки, когда after неизвестен

In [40]:
wm = WindowMaker(add_unchanged=True, left_window_size=2, right_window_size=2)
wm(test_df.head(100), mode='test')

before  \
sentence_id                                                        
0           0  Производится в Азии , Африке , России , Турции...   
            1                              с августа 2010 года .   
1           0         Теперь эти замки - туристические объекты .   
2           0           Проверено 26 августа 2012 . Архивировано   
            1       из первоисточника 19 октября 2012 . Интервью   
            2                                      Тараса Хтея .   
3           0                                       — С. 52 . С.   
            1  Карапетян Памятники армянской культуры в зоне ...   
4           0                           В 1902 году было открыто   
            1              только пешеходное движение по мосту .   
5           0            Проверено 1 августа 2013 . Архивировано   
            1             из первоисточника 1 августа 2013 . Ice   
            2                                   . Ice Age : Dawn   
            3   : Dawn of the Dinosaurs for PlayStation 2 ( 2009   
            4                                     ) ( англ . ) .   
6           0                        В качестве ГЭУ корабля была   
            1                                  выбрана дизельная   

                                                           after  different  
sentence_id                                                                  
0           0  Производится в Азии , Африке , России , Турции...      False  
            1                              с августа 2010 года .       True  
1           0         Теперь эти замки - туристические объекты .      False  
2           0           Проверено 26 августа 2012 . Архивировано       True  
            1       из первоисточника 19 октября 2012 . Интервью       True  
            2                                      Тараса Хтея .      False  
3           0                                       — С. 52 . С.       True  
            1  Карапетян Памятники армянской культуры в зоне ...      False  
4           0                           В 1902 году было открыто       True  
            1              только пешеходное движение по мосту .      False  
5           0            Проверено 1 августа 2013 . Архивировано       True  
            1             из первоисточника 1 августа 2013 . Ice       True  
            2                                   . Ice Age : Dawn       True  
            3   : Dawn of the Dinosaurs for PlayStation 2 ( 2009       True  
            4                                     ) ( англ . ) .      False  
6           0                        В качестве ГЭУ корабля была       True  
            1                                  выбрана дизельная      False

In [105]:
def _train_test_split(df: pd.DataFrame, idx_column : str, params):
    indices = np.array(list(set(df[idx_column].to_list())))
    train_indices, dev_indices = train_test_split(indices, **params)
    flt = df[idx_column].isin(train_indices)
    return df[flt], df[~flt]

In [37]:
sorted(_X_train_words.after.apply(len))[-20:]

[379,
 379,
 388,
 390,
 395,
 395,
 395,
 406,
 417,
 425,
 459,
 486,
 493,
 509,
 574,
 589,
 661,
 677,
 715,
 1003]

Удалим слишком длинные таргеты

In [107]:
train_df_cut = train_df.copy()
train_df_cut = train_df_cut[train_df_cut['before'].apply(len) < 400]
train_df_cut = train_df_cut[train_df_cut['after'].apply(len) < 400]

In [108]:
_X_train_words, _X_dev_words = _train_test_split(
    train_df_cut, 
    idx_column='sentence_id',
    params=dict(test_size=0.1, random_state=3)
)
_X_pretrain_words = _X_train_words.copy()
_X_pretrain_words.after = _X_pretrain_words.before

In [110]:
len(_X_dev_words) / len(_X_train_words)

0.11131697866811584

In [ ]:
%%time

wm = WindowMaker(add_unchanged=True, left_window_size=2, right_window_size=2)

train_df_windows = wm(_X_train_words)
dev_df_windows = wm(_X_dev_words)


In [ ]:
%%time

wm = WindowMaker(left_window_size=2, right_window_size=2)

train_df_windows = wm(_X_train_words)
dev_df_windows = wm(_X_dev_words)


In [ ]:
test_df_windows = wm(test_df, mode='test')

In [111]:
_X_pretrain_words.to_csv('X_pretrain_words_allennlp.tsv', sep='\t', header=False, columns=['before', 'after'], index=False)
_X_train_words.to_csv('X_train_words_allennlp.tsv', sep='\t', header=False, columns=['before', 'after'], index=False)
_X_dev_words.to_csv('X_dev_words_allennlp.tsv', sep='\t', header=False, columns=['before', 'after'], index=False)
test_df.to_csv('X_test_words_allennlp.tsv', sep='\t', header=False, columns=['before'], index=False)

In [93]:
source_tokenizer = WordTokenizer(
    word_splitter=CharacterWithTransWordSplitter(lowercase_characters=False), 
#     start_tokens=['<sos>'],
    end_tokens=['<eos>'],
)
target_tokenizer = WordTokenizer(
    word_splitter=CharacterWithTransWordSplitter(lowercase_characters=False),
    start_tokens=['<sos>'],
#     end_tokens=['<eos>'],
)
source_token_indexer =  {
    'tokens': SingleIdTokenIndexer(namespace='tokens')
}
target_token_indexer =  {
    'tokens': SingleIdTokenIndexer(namespace='target_tokens')
}
reader = Seq2SeqDatasetReader(
    source_tokenizer=source_tokenizer,
    target_tokenizer=target_tokenizer,
        source_token_indexers=source_token_indexer,
        target_token_indexers=target_token_indexer,
#         lazy=True
)
# X_dev = reader.read('X_dev_words_allennlp_10000.tsv')
# X_train = reader.read('X_train_words_allennlp_10000.tsv')
# X_pretrain = reader.read('X_pretrain_words_allennlp_10000.tsv')

X_train = reader.read('X_train_words_allennlp.tsv')
X_dev = reader.read('X_dev_words_allennlp.tsv')
X_pretrain = reader.read('X_pretrain_words_allennlp.tsv')

9515287it [06:51, 23151.27it/s]
9515287it [08:38, 18354.39it/s]
9515287it [05:57, 26613.22it/s]


In [ ]:
X_dev = reader.read('X_dev_words_allennlp.tsv')

In [ ]:
print(X_train[0])

In [113]:
vocab = Vocabulary.from_instances(X_train + X_pretrain + X_dev,
                                  min_count={'source_tokens': 1,
                                             'target_tokens': 1})

100%|██████████| 20089787/20089787 [03:04<00:00, 109138.82it/s]


In [114]:
vocab.print_statistics()



----Vocabulary Statistics----


Top 10 most frequent tokens in namespace 'tokens':
	Token: @start@		Frequency: 20089787
	Token: <eos>		Frequency: 20089787
	Token: @end@		Frequency: 20089787
	Token: о		Frequency: 8793627
	Token: а		Frequency: 6988017
	Token: е		Frequency: 6929056
	Token: и		Frequency: 6602483
	Token: н		Frequency: 5561417
	Token: р		Frequency: 4750800
	Token: с		Frequency: 4562374

Top 10 longest tokens in namespace 'tokens':
	Token: Heliand.comHelitrans		length: 20	Frequency: 2
	Token: Simutrans		length: 9	Frequency: 4
	Token: @start@		length: 7	Frequency: 20089787
	Token: latrans		length: 7	Frequency: 2
	Token: Intrans		length: 7	Frequency: 2
	Token: Motrans		length: 7	Frequency: 2
	Token: <eos>		length: 5	Frequency: 20089787
	Token: @end@		length: 5	Frequency: 20089787
	Token: trans		length: 5	Frequency: 25
	Token: о		length: 1	Frequency: 8793627

Top 10 shortest tokens in namespace 'tokens':
	Token: 외		length: 1	Frequency: 1
	Token: 休		length: 1	Frequency: 1
	Toke

In [115]:
VOCAB_DIR = "vocabulary2"

In [116]:

vocab.save_to_files(VOCAB_DIR)

In [49]:
vocab = Vocabulary.from_files(VOCAB_DIR)

In [50]:
vocab.get_vocab_size('tokens')

182

In [117]:
CUDA_DEVICE = 0
CUDA_DEVICES = list(range(8))
EN_EMBEDDING_DIM = 256
HIDDEN_DIM = 256
max_decoding_steps = 10
ZH_EMBEDDING_DIM = 256

In [118]:


try:
    del model
except:
    pass

try:
    del trainer
except:
    pass
try:
    del pretrainer
except:
    pass
    

torch.cuda.empty_cache()

In [119]:
# Скалярное произведение -- allennlp.modules.attention.dot_product_attention.DotProductAttention
# Аддитивное внимание -- allennlp.modules.attention.additive_attention.AdditiveAttention
# Мультипликативное внимание -- allennlp.modules.attention.bilinear_attention.BilinearAttention

def get_attentions(hidden_dim, activation = None):
    attentions = {
        'dotproduct' : DotProductAttention(),
        'additive' : AdditiveAttention(hidden_dim, hidden_dim),
        'bilinear' : BilinearAttention(hidden_dim, hidden_dim, activation=activation),
    }
    return attentions

def get_attention(name: str, hidden_dim: int, activation = None):
    return get_attentions(hidden_dim, activation)[name]

In [120]:
def get_encoder(name: str):
    if name == 'lstm':
        return PytorchSeq2SeqWrapper(
            torch.nn.LSTM(EN_EMBEDDING_DIM, HIDDEN_DIM, batch_first=True)
        )
    if name == 'transformer':
        return  StackedSelfAttentionEncoder(
            input_dim=EN_EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            projection_dim=128,
            feedforward_hidden_dim=128,
            num_layers=1, 
            num_attention_heads=8
        )
    assert False

In [121]:
def get_model(*, vocab, encoder_params, attention_params, max_decoding_steps):
    en_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                             embedding_dim=EN_EMBEDDING_DIM)
    encoder = get_encoder(**encoder_params)
#     encoder = PytorchSeq2SeqWrapper(
#         torch.nn.LSTM(EN_EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
#     encoder = StackedSelfAttentionEncoder(input_dim=EN_EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, projection_dim=128, feedforward_hidden_dim=128, num_layers=1, num_attention_heads=8)

    source_embedder = BasicTextFieldEmbedder({"tokens": en_embedding})

    # attention = LinearAttention(HIDDEN_DIM, HIDDEN_DIM, activation=Activation.by_name('tanh')())
    # attention = BilinearAttention(HIDDEN_DIM, HIDDEN_DIM)
#     attention = DotProductAttention()
    attention = get_attention(**attention_params)

    model = SimpleSeq2Seq(vocab, source_embedder, encoder, max_decoding_steps,
                          target_embedding_dim=ZH_EMBEDDING_DIM,
                          target_namespace='target_tokens',
                          attention=attention,
                          beam_size=8,
                          use_bleu=True)
    model.cuda(CUDA_DEVICE)
    
    return model


def get_trainer(*, model, X_train, X_test):
   
    optimizer = optim.Adam(model.parameters())
    iterator = BucketIterator(batch_size=1024, sorting_keys=[("source_tokens", "num_tokens")])

    iterator.index_with(vocab)

    trainer = Trainer(model=model,
                      optimizer=optimizer,
                      iterator=iterator,
                      train_dataset=X_train,
                      validation_dataset=X_test,
                      num_epochs=1,
                      cuda_device=CUDA_DEVICES)

    return trainer

def run_trainer(*, model,  trainer, epochs, X_test):
    for i in range(epochs):
        print('Epoch: {}'.format(i))
        trainer.train()

        predictor = SimpleSeq2SeqPredictor(model, reader)

        for instance in itertools.islice(X_test, 20):
            print('SOURCE:', ''.join(map(str, instance.fields['source_tokens'].tokens)))
            print('GOLD:', ''.join(map(str, instance.fields['target_tokens'].tokens)))
            print('PRED:', ''.join(map(str, predictor.predict_instance(instance)['predicted_tokens'])))
        print()
    return trainer

In [122]:
model = get_model(
    vocab=vocab,
    encoder_params=dict(
        name='transformer',
    ),
    attention_params=dict(
        name='dotproduct',
        hidden_dim=HIDDEN_DIM,
    ),
    max_decoding_steps=400
)

In [ ]:
pretrainer = get_trainer(model=model, X_train=X_pretrain[:10**6], X_test=X_dev[:100])
run_trainer(model=model, trainer=pretrainer, epochs=100, X_test=X_dev)

Epoch: 0


loss: 2.7169 ||: 100%|██████████| 123/123 [01:24<00:00,  1.46it/s]
BLEU: 0.0379, loss: 1.9879 ||: 100%|██████████| 1/1 [00:00<00:00,  3.04it/s]


SOURCE: @start@По<eos>@end@
GOLD: @start@<sos>По@end@
PRED: <sos>и
SOURCE: @start@состоянию<eos>@end@
GOLD: @start@<sos>состоянию@end@
PRED: <sos>порононе
SOURCE: @start@на<eos>@end@
GOLD: @start@<sos>на@end@
PRED: <sos>и
SOURCE: @start@1862 год<eos>@end@
GOLD: @start@<sos>тысяча восемьсот шестьдесят второй год@end@
PRED: <sos>порононе
SOURCE: @start@.<eos>@end@
GOLD: @start@<sos>.@end@
PRED: <sos>.
SOURCE: @start@Севернее<eos>@end@
GOLD: @start@<sos>Севернее@end@
PRED: <sos>порононе
SOURCE: @start@Дудинки<eos>@end@
GOLD: @start@<sos>Дудинки@end@
PRED: <sos>порона
SOURCE: @start@и<eos>@end@
GOLD: @start@<sos>и@end@
PRED: <sos>в
SOURCE: @start@северо<eos>@end@
GOLD: @start@<sos>северо@end@
PRED: <sos>пороно
SOURCE: @start@-<eos>@end@
GOLD: @start@<sos>-@end@
PRED: <sos>.
SOURCE: @start@восточнее<eos>@end@
GOLD: @start@<sos>восточнее@end@
PRED: <sos>порононе
SOURCE: @start@Белочи<eos>@end@
GOLD: @start@<sos>Белочи@end@
PRED: <sos>пороно
SOURCE: @start@,<eos>@end@
GOLD: @start@<sos>,@end@

loss: 1.7331 ||: 100%|██████████| 123/123 [01:11<00:00,  1.72it/s]
BLEU: 0.0673, loss: 1.8279 ||: 100%|██████████| 1/1 [00:00<00:00,  2.83it/s]


SOURCE: @start@По<eos>@end@
GOLD: @start@<sos>По@end@
PRED: <sos>на
SOURCE: @start@состоянию<eos>@end@
GOLD: @start@<sos>состоянию@end@
PRED: <sos>поранина
SOURCE: @start@на<eos>@end@
GOLD: @start@<sos>на@end@
PRED: <sos>на
SOURCE: @start@1862 год<eos>@end@
GOLD: @start@<sos>тысяча восемьсот шестьдесят второй год@end@
PRED: <sos>порана
SOURCE: @start@.<eos>@end@
GOLD: @start@<sos>.@end@
PRED: <sos>.
SOURCE: @start@Севернее<eos>@end@
GOLD: @start@<sos>Севернее@end@
PRED: <sos>порана
SOURCE: @start@Дудинки<eos>@end@
GOLD: @start@<sos>Дудинки@end@
PRED: <sos>порана
SOURCE: @start@и<eos>@end@
GOLD: @start@<sos>и@end@
PRED: <sos>в
SOURCE: @start@северо<eos>@end@
GOLD: @start@<sos>северо@end@
PRED: <sos>поста
SOURCE: @start@-<eos>@end@
GOLD: @start@<sos>-@end@
PRED: <sos>-
SOURCE: @start@восточнее<eos>@end@
GOLD: @start@<sos>восточнее@end@
PRED: <sos>постона
SOURCE: @start@Белочи<eos>@end@
GOLD: @start@<sos>Белочи@end@
PRED: <sos>поста
SOURCE: @start@,<eos>@end@
GOLD: @start@<sos>,@end@
PRED

...

loss: 0.8317 ||: 100%|██████████| 123/123 [01:12<00:00,  1.70it/s]
BLEU: 0.5975, loss: 0.8811 ||: 100%|██████████| 1/1 [00:00<00:00,  2.55it/s]


SOURCE: @start@По<eos>@end@
GOLD: @start@<sos>По@end@
PRED: <sos>По
SOURCE: @start@состоянию<eos>@end@
GOLD: @start@<sos>состоянию@end@
PRED: <sos>состоянию
SOURCE: @start@на<eos>@end@
GOLD: @start@<sos>на@end@
PRED: <sos>на
SOURCE: @start@1862 год<eos>@end@
GOLD: @start@<sos>тысяча восемьсот шестьдесят второй год@end@
PRED: <sos>1998 год
SOURCE: @start@.<eos>@end@
GOLD: @start@<sos>.@end@
PRED: <sos>.
SOURCE: @start@Севернее<eos>@end@
GOLD: @start@<sos>Севернее@end@
PRED: <sos>Севернее
SOURCE: @start@Дудинки<eos>@end@
GOLD: @start@<sos>Дудинки@end@
PRED: <sos>Дудинки
SOURCE: @start@и<eos>@end@
GOLD: @start@<sos>и@end@
PRED: <sos>и
SOURCE: @start@северо<eos>@end@
GOLD: @start@<sos>северо@end@
PRED: <sos>северо
SOURCE: @start@-<eos>@end@
GOLD: @start@<sos>-@end@
PRED: <sos>-
SOURCE: @start@восточнее<eos>@end@
GOLD: @start@<sos>восточнее@end@
PRED: <sos>восточнее
SOURCE: @start@Белочи<eos>@end@
GOLD: @start@<sos>Белочи@end@
PRED: <sos>Белочи
SOURCE: @start@,<eos>@end@
GOLD: @start@<sos>,

loss: 0.1750 ||: 100%|██████████| 123/123 [01:11<00:00,  1.72it/s]
BLEU: 0.7517, loss: 0.9443 ||: 100%|██████████| 1/1 [00:00<00:00,  2.71it/s]


SOURCE: @start@По<eos>@end@
GOLD: @start@<sos>По@end@
PRED: <sos>По
SOURCE: @start@состоянию<eos>@end@
GOLD: @start@<sos>состоянию@end@
PRED: <sos>состоянию
SOURCE: @start@на<eos>@end@
GOLD: @start@<sos>на@end@
PRED: <sos>на
SOURCE: @start@1862 год<eos>@end@
GOLD: @start@<sos>тысяча восемьсот шестьдесят второй год@end@
PRED: <sos>1862 год
SOURCE: @start@.<eos>@end@
GOLD: @start@<sos>.@end@
PRED: <sos>.
SOURCE: @start@Севернее<eos>@end@
GOLD: @start@<sos>Севернее@end@
PRED: <sos>Севернее
SOURCE: @start@Дудинки<eos>@end@
GOLD: @start@<sos>Дудинки@end@
PRED: <sos>Дудинки
SOURCE: @start@и<eos>@end@
GOLD: @start@<sos>и@end@
PRED: <sos>и
SOURCE: @start@северо<eos>@end@
GOLD: @start@<sos>северо@end@
PRED: <sos>северо
SOURCE: @start@-<eos>@end@
GOLD: @start@<sos>-@end@
PRED: <sos>-
SOURCE: @start@восточнее<eos>@end@
GOLD: @start@<sos>восточнее@end@
PRED: <sos>восточнее
SOURCE: @start@Белочи<eos>@end@
GOLD: @start@<sos>Белочи@end@
PRED: <sos>Белочи
SOURCE: @start@,<eos>@end@
GOLD: @start@<sos>,

loss: 0.0531 ||:  71%|███████   | 87/123 [00:57<00:15,  2.30it/s]

In [71]:
trainer = get_trainer(model=model, X_train=X_train, X_test=X_dev[:100])
run_trainer(model=model, trainer=trainer, epochs=200, X_test=X_dev)

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


Epoch: 0


loss: 5.0082 ||: 100%|██████████| 3/3 [00:01<00:00,  2.48it/s]
BLEU: 0.0000, loss: 4.3749 ||: 100%|██████████| 1/1 [00:00<00:00,  2.89it/s]


SOURCE: @start@Оснащались<eos>@end@
GOLD: @start@<sos>Оснащались@end@
PRED: 
SOURCE: @start@латными<eos>@end@
GOLD: @start@<sos>латными@end@
PRED: 
SOURCE: @start@рукавицами<eos>@end@
GOLD: @start@<sos>рукавицами@end@
PRED: 
SOURCE: @start@и<eos>@end@
GOLD: @start@<sos>и@end@
PRED: 
SOURCE: @start@сабатонами<eos>@end@
GOLD: @start@<sos>сабатонами@end@
PRED: 
SOURCE: @start@с<eos>@end@
GOLD: @start@<sos>с@end@
PRED: 
SOURCE: @start@не<eos>@end@
GOLD: @start@<sos>не@end@
PRED: 
SOURCE: @start@длинными<eos>@end@
GOLD: @start@<sos>длинными@end@
PRED: 
SOURCE: @start@носками<eos>@end@
GOLD: @start@<sos>носками@end@
PRED: 
SOURCE: @start@.<eos>@end@
GOLD: @start@<sos>.@end@
PRED: 
SOURCE: @start@В<eos>@end@
GOLD: @start@<sos>В@end@
PRED: 
SOURCE: @start@конце<eos>@end@
GOLD: @start@<sos>конце@end@
PRED: 
SOURCE: @start@1811 года<eos>@end@
GOLD: @start@<sos>тысяча восемьсот одиннадцатого года@end@
PRED: 
SOURCE: @start@,<eos>@end@
GOLD: @start@<sos>,@end@
PRED: 
SOURCE: @start@вследствие<eos>

loss: 4.0824 ||: 100%|██████████| 3/3 [00:01<00:00,  2.38it/s]
BLEU: 0.0000, loss: 3.5703 ||: 100%|██████████| 1/1 [00:00<00:00,  2.99it/s]


...

SOURCE: @start@Оснащались<eos>@end@
GOLD: @start@<sos>Оснащались@end@
PRED: <sos>уо то тн
SOURCE: @start@латными<eos>@end@
GOLD: @start@<sos>латными@end@
PRED: <sos>уоресн
SOURCE: @start@рукавицами<eos>@end@
GOLD: @start@<sos>рукавицами@end@
PRED: <sos>уо тосе
SOURCE: @start@и<eos>@end@
GOLD: @start@<sos>и@end@
PRED: <sos>н
SOURCE: @start@сабатонами<eos>@end@
GOLD: @start@<sos>сабатонами@end@
PRED: <sos>уо тосе
SOURCE: @start@с<eos>@end@
GOLD: @start@<sos>с@end@
PRED: <sos>н
SOURCE: @start@не<eos>@end@
GOLD: @start@<sos>не@end@
PRED: <sos>се
SOURCE: @start@длинными<eos>@end@
GOLD: @start@<sos>длинными@end@
PRED: <sos>уо тнсн
SOURCE: @start@носками<eos>@end@
GOLD: @start@<sos>носками@end@
PRED: <sos>уоресн
SOURCE: @start@.<eos>@end@
GOLD: @start@<sos>.@end@
PRED: <sos>ы
SOURCE: @start@В<eos>@end@
GOLD: @start@<sos>В@end@
PRED: <sos>н
SOURCE: @start@конце<eos>@end@
GOLD: @start@<sos>конце@end@
PRED: <sos> тол
SOURCE: @start@1811 года<eos>@end@
GOLD: @start@<sos>тысяча восемьсот одиннадца

loss: 1.6330 ||: 100%|██████████| 3/3 [00:01<00:00,  2.38it/s]
BLEU: 0.0128, loss: 1.4814 ||: 100%|██████████| 1/1 [00:02<00:00,  2.72s/it]


SOURCE: @start@Оснащались<eos>@end@
GOLD: @start@<sos>Оснащались@end@
PRED: <sos>уо то тнсн
SOURCE: @start@латными<eos>@end@
GOLD: @start@<sos>латными@end@
PRED: <sos>уо тнсн
SOURCE: @start@рукавицами<eos>@end@
GOLD: @start@<sos>рукавицами@end@
PRED: <sos>уо тоснсе
SOURCE: @start@и<eos>@end@
GOLD: @start@<sos>и@end@
PRED: <sos>н
SOURCE: @start@сабатонами<eos>@end@
GOLD: @start@<sos>сабатонами@end@
PRED: <sos>уо торнсе
SOURCE: @start@с<eos>@end@
GOLD: @start@<sos>с@end@
PRED: <sos>н
SOURCE: @start@не<eos>@end@
GOLD: @start@<sos>не@end@
PRED: <sos>се
SOURCE: @start@длинными<eos>@end@
GOLD: @start@<sos>длинными@end@
PRED: <sos>уорнснсн
SOURCE: @start@носками<eos>@end@
GOLD: @start@<sos>носками@end@
PRED: <sos>уо тнсн
SOURCE: @start@.<eos>@end@
GOLD: @start@<sos>.@end@
PRED: <sos>ы
SOURCE: @start@В<eos>@end@
GOLD: @start@<sos>В@end@
PRED: <sos>н
SOURCE: @start@конце<eos>@end@
GOLD: @start@<sos>конце@end@
PRED: <sos>уо то
SOURCE: @start@1811 года<eos>@end@
GOLD: @start@<sos>тысяча восемьсот

loss: 1.6068 ||:  67%|██████▋   | 2/3 [00:01<00:00,  1.90it/s]


KeyboardInterrupt: 